In [ ]:
# 데이터 js로 변환

import pandas as pd
import json

# --- 장르 리스트 변환 ---
def tidy_genres(gstr):
    if pd.isnull(gstr):
        return []
    return [g.strip() for g in str(gstr).split(',') if g.strip()]

# --- 문자열을 불린으로 변환 ---
def boolify(val):
    sval = str(val).lower().strip()
    return sval in ['true', '1', 'yes', 'y']

# --- 행(row)을 웹툰 객체로 변환 ---
def to_webtoon_obj(row):
    return {
        'id': int(row['id']) if not pd.isnull(row['id']) else 0,
        'title': str(row['title']),
        'author': str(row['author']),
        'genre': tidy_genres(row['genre']),
        'description': str(row['description']) if not pd.isnull(row['description']) else '',
        'rating': float(row['rating']) if not pd.isnull(row['rating']) else 0,
        'completed': boolify(row['completed']),
        'age': str(row['age']) if not pd.isnull(row['age']) else '',
        'free': boolify(row['free']),
        'link': str(row['link']) if not pd.isnull(row['link']) else '#',
        'img': str(row['thumbnail_url']) if not pd.isnull(row['thumbnail_url']) else ''
    }

# --- CSV 읽기 (UTF-8 확정) ---
csv_path = './data/naver.csv'
df = pd.read_csv(csv_path, encoding='utf-8')

# --- 각 행을 JS용 객체로 변환 ---
webtoons = [to_webtoon_obj(row) for _, row in df.iterrows()]

# --- 장르 / 연령 추출 ---
all_genres = sorted({g for w in webtoons for g in w['genre']})
all_ages = sorted({w['age'] for w in webtoons if w['age']})

# --- JS 파일로 저장 ---
data = {'webtoons': webtoons, 'genres': all_genres, 'ages': all_ages}
with open('webtoons_data.js', 'w', encoding='utf-8') as f:
    f.write('const webtoonsData = ')
    json.dump(data, f, ensure_ascii=False, indent=2)
    f.write(';')

print(" webtoons_data.js 파일이 성공적으로 생성되었습니다. (UTF-8 인코딩 사용)")


✅ webtoons_data.js 파일이 성공적으로 생성되었습니다. (UTF-8 인코딩 사용)


In [ ]:
# 썸네일 크롤링

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import time

# 1. naver.csv 파일 로드
try:
    df = pd.read_csv('naver.csv')
except FileNotFoundError:
    print("'naver.csv' 파일을 찾을 수 없습니다. 파일이 코드와 동일한 디렉토리에 있는지 확인하세요.")
    exit()

def get_thumbnail_with_selenium(title_to_search):
    """
    Selenium을 사용하여 웹툰 제목으로 페이지에 접속한 후,
    class가 'Poster__image--d9XTI'인 img 태그의 src 값을 추출합니다.
    """
    # 2. 제목으로 데이터프레임에서 link 주소 찾기
    webtoon_data = df[df['title'] == title_to_search]
    if webtoon_data.empty:
        return f"'{title_to_search}'에 해당하는 웹툰을 찾을 수 없습니다."

    webtoon_link = webtoon_data.iloc[0]['link']

    # 3. Selenium 설정 및 웹 드라이버 실행
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # 브라우저 창을 띄우지 않고 백그라운드에서 실행
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    
    # 웹 드라이버 서비스 실행 (최신 Selenium 방식)
    service = Service()
    driver = webdriver.Chrome(service=service, options=chrome_options)

    try:
        # 4. 웹툰 페이지로 이동
        driver.get(webtoon_link)
        
        # 페이지가 완전히 로드될 때까지 잠시 대기 (필요에 따라 시간 조절)
        time.sleep(2)

        # 5. 지정된 class 이름으로 img 요소 찾기
        img_element = driver.find_element(By.CSS_SELECTOR, "img.Poster__image--d9XTI")
        
        # 'src' 속성 값 가져오기
        thumbnail_url = img_element.get_attribute('src')
        return thumbnail_url

    except NoSuchElementException:
        return "페이지에서 'Poster__image--d9XTI' 클래스를 가진 이미지를 찾지 못했습니다."
    except Exception as e:
        return f"오류가 발생했습니다: {e}"
    finally:
        # 6. 작업이 끝나면 드라이버 종료
        driver.quit()

# --- 함수 사용 예시 ---
search_title = '가비지타임'
selenium_thumbnail_url = get_thumbnail_with_selenium(search_title)

print(f"웹툰 제목: {search_title}")
print(f"찾은 최종 이미지 URL (Selenium): {selenium_thumbnail_url}")

search_title_2 = '가난을 등에 업은 소녀'
selenium_thumbnail_url_2 = get_thumbnail_with_selenium(search_title_2)

print(f"\n웹툰 제목: {search_title_2}")
print(f"찾은 최종 이미지 URL (Selenium): {selenium_thumbnail_url_2}")

